In [ ]:
!nvidia-smi

In [1]:
%matplotlib inline

In [ ]:
!pip install -r requirements.txt

In [ ]:

!pip install tensorboard

# uso y pruebas

In [2]:
!python classifier.py --help
# !python classifier_bert.py --help
# !python classifier.py --help

usage: classifier.py [-h] [-g GPU] [-f MODEL_FILE] [-t TRAIN_CFG]
                     [-a MASK_CFG] [-l LABEL_INDEX] [-s SAVE_MODEL]
                     model_version {hhar,motion,uci,shoaib} {10_100,20_120}

PyTorch LIMU-BERT Model

positional arguments:
  model_version         Model config
  {hhar,motion,uci,shoaib}
                        Dataset name
  {10_100,20_120}       Dataset version

optional arguments:
  -h, --help            show this help message and exit
  -g GPU, --gpu GPU     Set specific GPU
  -f MODEL_FILE, --model_file MODEL_FILE
                        Pretrain model file
  -t TRAIN_CFG, --train_cfg TRAIN_CFG
                        Training config json file path
  -a MASK_CFG, --mask_cfg MASK_CFG
                        Mask strategy json file path
  -l LABEL_INDEX, --label_index LABEL_INDEX
                        Label Index
  -s SAVE_MODEL, --save_model SAVE_MODEL
                        The saved model name
usage: classifier.py [-h] [-g GPU] [-f MODEL_FILE] 

In [ ]:
!python pretrain.py v1 uci 20_120 -s limu_v1 

In [ ]:
!python embedding.py v1 uci 20_120 -f limu_v1


In [ ]:
!python embedding.py "BERT" "hhar" "20_120" -g 0

# utils.py linea 411 msj de erro